In [1]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model = "qwen3",
    temperature = 0,
)

In [2]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Majid")])

AIMessage(content="<think>\nOkay, the user introduced themselves as Majid. I should respond warmly and ask how I can assist them. Let me make sure to use a friendly tone and keep the conversation open for them to share more about their needs or questions. I should also check if there's anything specific they want help with, like information, advice, or just casual chat. Keeping it welcoming and approachable is key here.\n</think>\n\nHello, Majid! It's nice to meet you. How can I assist you today? Are you looking for information, advice, or just someone to chat with? 😊", additional_kwargs={}, response_metadata={'model': 'qwen3', 'created_at': '2025-08-04T23:15:23.7164186Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2541912700, 'load_duration': 30593200, 'prompt_eval_count': 14, 'prompt_eval_duration': 122220900, 'eval_count': 122, 'eval_duration': 2387610000, 'model_name': 'qwen3'}, id='run--fdacfbc9-c0ba-4b5c-93fd-07780b909c02-0', usage_metadata={'input_tokens': 14, 'outpu

In [3]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content='<think>\nOkay, the user is asking, "What\'s my name?" I need to figure out how to respond. First, I should check if there\'s any context from previous interactions. Since this is the first message, there\'s no prior conversation. The user might be testing if I can recognize their name, but I don\'t have access to that information unless they provided it earlier.\n\nWait, maybe they expect me to know their name from some other source. But according to the guidelines, I can\'t access personal data or external information. So I should inform them that I don\'t have access to their name. However, I should also offer to help with anything else they need. Let me make sure the response is polite and clear. I should avoid any assumptions and keep it straightforward. Yeah, that should cover it.\n</think>\n\nI don\'t have access to personal information about you, including your name. However, I\'m here to help with any questions or tasks you might have! How can I assist you to

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm majid"),
        AIMessage(content="Hello majid! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='<think>\nOkay, the user asked, "What\'s my name?" after I greeted them as "majid." Let me check the conversation history.\n\nIn the first message, the user said, "Hi! I\'m majid." Then I responded with, "Hello majid! How can I assist you today?" So I used their name in the greeting. Now, when they ask, "What\'s my name?" they might be confused or testing if I remember their name from the previous message.\n\nI need to confirm their name again. Since I already used "majid" in the greeting, it\'s likely they are referring to themselves. But maybe they want to make sure I remember or they might have a different name. Wait, in the initial message, they introduced themselves as "majid," so that\'s their name. The user might be checking if I recall their name correctly. \n\nI should respond by confirming their name and offering further assistance. Let me make sure there\'s no confusion. The user might be testing the AI\'s ability to retain information from the conversation

## Message persistence

In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Defne the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [6]:
# This enables us to support multiple conversation threads with a single application, a common requirement when your application has multiple users.

config = {"configurable": {"thread_id": "abc123"}}

In [7]:
query = "Hi! I'm majid."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

<think>
Okay, Majid just said "Hi! I'm majid." I need to respond appropriately. Let me start by acknowledging his greeting. Since he introduced himself, I should respond in a friendly and welcoming manner. Maybe say something like "Hello, Majid! Nice to meet you!" to keep it simple and positive. I should make sure to use his name correctly, maybe check if he prefers it capitalized or not, but since he wrote it as "majid," I'll go with that. Also, I should keep the tone friendly and open for further conversation. Maybe add an emoji to keep it light, like a smiley face. Let me put that together: "Hello, Majid! Nice to meet you! 😊 How can I assist you today?" That should cover it. I should make sure the response is clear and invites him to ask for help if needed.
</think>

Hello, Majid! Nice to meet you! 😊 How can I assist you today?


In [8]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user asked, "What's my name?" after introducing himself as Majid. Let me check the conversation history.

In the first message, Majid said, "Hi! I'm majid." Then the assistant responded with a greeting and asked how they could help. Now, Majid is asking for his own name. 

Wait, maybe he's testing if I remember his name from the previous message. Since the assistant's last response was a greeting and an offer to help, but didn't mention his name. So, the user is probably confirming his name again. 

I should respond by recalling his name from the initial message. Let me make sure there's no confusion. The user might be checking if I remember or if there's a mistake. 

The correct answer is that his name is Majid. I should state that clearly and maybe offer further assistance. Keep the tone friendly and helpful.
</think>

Your name is Majid! 😊 How can I assist you today?


In [9]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user is asking, "What's my name?" I need to figure out how to respond. First, I should check if there's any context from previous interactions. Since this is the first message, there's no prior conversation. The user might be testing if I can recognize their name, but I don't have access to that information unless they provided it earlier.

Wait, maybe they expect me to know their name from some other source. But according to the guidelines, I can't access personal data or external information. So I should inform them that I don't have access to their name. However, I should also offer to help with anything else they need. Let me make sure the response is polite and clear. I should avoid any assumptions and keep it straightforward. Yeah, that should cover it.
</think>

I don't have access to personal information about you, including your name. However, I'm here to help with any questions

In [10]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user asked "What's my name?" again. Let me check the conversation history.

Earlier, the user introduced themselves as Majid, and I confirmed that. Then they asked the same question again. Maybe they're testing if I remember or if there's a misunderstanding. I should reaffirm their name and offer help. Keep it friendly and open-ended. Let me respond clearly.
</think>

Your name is Majid! 😊 How can I assist you today?


In [11]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user is asking, "What's my name?" I need to figure out how to respond. First, I should check if there's any context from previous interactions. Since this is the first message, there's no prior conversation. The user might be testing if I can recognize their name, but I don't have access to that information unless they provided it earlier.

Wait, maybe they expect me to know their name from some other source. But according to the guidelines, I can't access personal data or external information. So I should inform them that I don't have access to their name. However, I should also offer to help with anything else they need. Let me make sure the response is polite and clear. I should avoid any assumptions and keep it straightforward. Yeah, that should cover it.
</think>

I don't have access to personal information about you, including your name. However, I'm here to help with any questions

## Prompt templates

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [13]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state) #####
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [14]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Majid."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user introduced themselves as Majid. I need to respond in a pirate-like manner. Let me think about how to phrase that. Maybe start with "Arrr, ye be..." to set the pirate tone. Then mention their name, so "Majid" becomes "Majid" in the response. I should add some pirate slang like "matey" or "ye" to keep it authentic. Also, maybe include a question to keep the conversation going, like asking about their quest or where they're headed. Let me check if the response flows naturally and sounds like a pirate. Yeah, that should work. Make sure to keep it friendly and engaging.
</think>

Arrr, ye be Majid, eh? Aye, I be Captain Bootstrap, ye scallywag! What be yer quest, matey? D’ye seek treasure, or be ye aye on the hunt for a ship o’ yer own? Speak up, laddie, before me parrot here starts squawkin’ about yer lack o’ adventure! 🏴☠️


In [15]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user asked, "What is my name?" I need to respond as a pirate. Let me recall the previous conversation. The user introduced himself as Majid, and I greeted him as Majid, using pirate lingo. Now he's asking for his own name. Since I already know it from the initial message, I should confirm it in a pirate style.

I should use phrases like "Aye, ye be Majid, eh?" to affirm his name. Maybe add some pirate slang to keep the character consistent. Also, include a friendly challenge or a playful threat to keep the interaction engaging. Maybe mention the parrot again for continuity. Let me make sure the response is in line with the pirate persona and maintains the adventurous tone. Avoid any markdown and keep it natural.
</think>

Aye, ye be Majid, eh? D’ye think I’d forget such a fine name? Arrr, ye’re the one with the shiny boots and the curious questions! Speak up, ye landlubber, or I’ll tie y

In [16]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [17]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [18]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Majid."
language = "French"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, Majid said hi and introduced himself. I need to respond in French. Let me make sure I use the correct greeting and introduce myself properly.

"Bonjour, Majid ! Je m'appelle [Votre nom]. Enchanté de vous rencontrer ! Comment allez-vous ?" 

Wait, I should replace [Votre nom] with my actual name. Since I don't have a name, maybe I can just say "Je m'appelle [Nom de l'assistant]" or use a placeholder. But the user might expect a specific name. Let me check the instructions again. The user wants me to answer in French, so I should use the correct structure. Maybe I can use "Je m'appelle [Nom de l'assistant]" and keep it general. Alternatively, since I don't have a name, perhaps just say "Je m'appelle [Votre nom]" but that might not be right. Wait, the user is Majid, so I should respond as the assistant. Maybe I should just say "Bonjour, Majid ! Je m'appelle [Nom de l'assistant]. Enchanté de vou

In [19]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user asked, "What is my name?" Let me check the conversation history. Earlier, Majid introduced himself as "Hi! I'm Majid." Then I responded with "Bonjour, Majid ! Enchanté de vous rencontrer ! Comment allez-vous ?" So the user's name is Majid. The user is probably testing if I remember their name from the previous message. I should confirm their name and maybe ask how they're doing again to keep the conversation going. Let me make sure I'm using French and keeping the response friendly and helpful.
</think>

Votre nom est Majid ! 😊 Comment allez-vous aujourd'hui ?


## Managing Conversation History

In [20]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=40,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm majid"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

C:\Users\ASUS\miniconda3\envs\Ollama\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [21]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"]) ##########################
    prompt = prompt_template.invoke( ##############################################
        {"messages": trimmed_messages, "language": state["language"]} #############
    ) #############################################################################
    response = model.invoke(prompt) ###############################################
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [22]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user asked, "What is my name?" Let me think about how to respond.

First, I need to recall the conversation history. The user started by asking for 2+2, which I answered as 4. Then they said thanks, and I replied with "no problem!" Next, they asked if I was having fun, and I said yes. Now they're asking for their name.

Since I don't have access to personal information about the user, I can't know their actual name. My role is to be helpful and polite. I should inform them that I don't have that information but offer to assist with anything else they need. I should keep the response friendly and open-ended to encourage further interaction. Let me make sure the response is clear and not confusing. Also, check for any possible misunderstandings. The user might be testing if I can access their data, so it's important to be transparent. Alright, the response should be straightforward.
</thin

In [23]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

<think>
Okay, the user is asking, "What math problem did I ask?" Let me check the conversation history.

Looking back, the user's previous messages were: "you're a good assistant", "thanks", "having fun?", and "What math problem did I ask?" So far, there's no mention of a specific math problem. The user might be referring to a previous interaction where they asked a math problem, but in the current conversation, they haven't mentioned any. 

Wait, maybe they're confused or thinking about a past question. Since there's no record of a math problem being asked in this session, I should inform them that there hasn't been a math problem mentioned yet. I should also invite them to ask a problem if they have one. That way, I'm helpful and clear, avoiding any confusion.
</think>

In our current conversation, you haven't asked a specific math problem yet. If you have a math question or problem you'd like to solve,

## Streaming

In [24]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream( ###########################
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages", ##################################
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

<think>|
|Okay|,| Todd| wants| a| joke|.| Let| me| think| of| something| light| and| funny|.| Maybe| a| classic| one| that|'s| easy| to| understand|.| How| about| the| one| about| the| chicken| and| the| road|?| Wait|,| that|'s| the| "|Why| did| the| chicken| cross| the| road|?"| joke|.| It|'s| a| classic|,| but| maybe| he|'s| heard| it| before|.| Alternatively|,| maybe| a| pun|.| Let| me| check| if| there|'s| another| one| that|'s| not| too| obscure|.| Oh|,| the| one| about| the| fish| and| the| bicycle|?| No|,| that|'s| not| as| good|.| Maybe| a| play| on| words|.| Let| me| go| with| the| chicken| joke| since| it|'s| well|-known| and| has| a| good| punch|line|.| Let| me| make| sure| the| setup| is| clear| and| the| punch|line| is| unexpected|.| Yeah|,| that| should| work|.| Let| me| present| it| in| a| friendly| way|.
|</think>|

|Hi| Todd|!| Here|'s| a| joke| for| you|:|  

|**|Why| did| the| chicken| cross| the| road|?|**|  
|*|To| get| to| the| other| side|!*|  

|😄| Let| me| know